In [6]:
%load_ext autoreload
%autoreload 2

# Path to the main CAMELS folder
CAMELS_DIR = "/home/jhbai/ealstm_regional_modeling/data/CAMELS/"

# Path to the main directory of this repository
BASE_CODE_DIR = "/home/jhbai/ealstm_regional_modeling"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [73]:
# Imports
import json
import torch
import pickle
import sys
from collections import defaultdict
from pathlib import Path

import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from bokeh.sampledata import us_states

sys.path.append(BASE_CODE_DIR)
from main import Model
from papercode import datautils
from papercode.clusterutils import get_silhouette_scores, get_clusters, get_label_2_color, get_variance_reduction
from papercode.datasets import CamelsTXT
from papercode.datautils import load_attributes
from papercode.plotutils import get_shape_collections
from papercode.utils import get_basin_list

# get polygon points of only the CONUS states
us_states = us_states.data.copy()
del us_states["HI"]
del us_states["AK"]

In [8]:
# Convert file/folder path to pathlib Path objects
CAMELS_DIR = Path(CAMELS_DIR)
CODE_DIR = Path(BASE_CODE_DIR)

# Not really of importance, but need to be defined for the data set class.
VAL_START = pd.to_datetime('01101989', format='%d%m%Y')
VAL_END = pd.to_datetime('30091999', format='%d%m%Y')

In [4]:
CAMELS_DIR

PosixPath('/home/jhbai/ealstm_regional_modeling/data/CAMELS')

In [5]:
basins = get_basin_list()

In [7]:
#datautils.add_camels_attributes(CAMELS_DIR)

In [53]:
# draw the maps
from shapely.ops import unary_union
from shapely.geometry import Polygon
import folium
from pyproj import CRS, Transformer

shapefile = datautils.load_shape(CAMELS_DIR)

In [56]:

incrs = CRS.from_proj4("+proj=aea +lon_0=-96 +lat_1=29.5 +lat_2=45.5 +lat_0=23 +datum=NAD83")
crs_4326 = CRS.from_epsg(4326)
transformer = Transformer.from_crs(incrs, crs_4326)

In [71]:
m = folium.Map(width=650, height=500, zoom_start=4, 
                location=[41.960781097662604, -78.16963195732737],              
                tiles='Stamen Terrain')
shapefile1=shapefile[0:200]
new_shape=datautils.create_shape(shapefile, '02422500',transformer)
folium.GeoJson(new_shape,name='02046000',tooltip='02046000').add_to(m)

In [72]:
m

In [27]:
import re

filterlist=datautils.list_overlap(new_shape,CODE_DIR,0.125)
new_shape.area

0.029468126957630274

In [65]:
result=datautils.find_intersect(new_shape,filterlist)
result['wprec'] = result['weight']*result['precipitation']

in1980=result[(result['Year']==1980)]
g=in1980.groupby( ['Year', 'Mnth', 'Day'])
final=g.sum()

/home/jhbai/ealstm_regional_modeling/data/east/data_37.0625_-77.8125
/home/jhbai/ealstm_regional_modeling/data/east/data_36.9375_-77.9375
/home/jhbai/ealstm_regional_modeling/data/east/data_37.0625_-77.5625
/home/jhbai/ealstm_regional_modeling/data/east/data_37.1875_-77.9375
/home/jhbai/ealstm_regional_modeling/data/east/data_37.0625_-77.6875
/home/jhbai/ealstm_regional_modeling/data/east/data_37.1875_-77.8125
/home/jhbai/ealstm_regional_modeling/data/east/data_37.1875_-77.5625
/home/jhbai/ealstm_regional_modeling/data/east/data_36.9375_-77.6875
/home/jhbai/ealstm_regional_modeling/data/east/data_36.9375_-77.8125
/home/jhbai/ealstm_regional_modeling/data/east/data_36.9375_-77.5625
/home/jhbai/ealstm_regional_modeling/data/east/data_37.0625_-77.9375
/home/jhbai/ealstm_regional_modeling/data/east/data_37.1875_-77.6875


precipitation  max_temp  min_temp  wind_speed  weight  \
Year Mnth Day                                                          
1980 1    1             0.00    125.25    -48.83       61.84     1.0   
          2             0.00    116.03    -50.63      107.10     1.0   
          3             0.00    101.76    -82.19       44.90     1.0   
          4           120.88     72.65    -60.15       51.61     1.0   
          5           261.09     28.22    -40.66       93.35     1.0   
          6            10.71     19.42   -114.32       90.32     1.0   
          7             6.82     54.34   -113.16       54.40     1.0   
          8             0.00     71.10     -2.40       24.25     1.0   
          9             0.00     75.10      7.85       36.23     1.0   
          10            0.00     53.32    -75.18       75.38     1.0   
          11          308.60    112.29    -40.98       99.56     1.0   
          12           31.52    181.79      6.33       44.25     1.0   
          13           20.24     91.16    -21.83      102.79     1.0   
          14          312.89     59.94    -10.79       37.45     1.0   
          15           51.93    132.76     34.49      116.64     1.0   
          16            0.00    172.33    -16.41       84.42     1.0   
          17           77.70    148.32     -7.35       54.05     1.0   
          18          333.91    112.31     25.58       14.76     1.0   
          19           10.09    113.32     -7.74       91.02     1.0   
          20            1.35    147.58    -27.70       63.44     1.0   
          21            1.12    143.10    -52.63       55.05     1.0   
          22          162.45    124.51    -23.87       57.88     1.0   
          23           41.26    129.07     43.52       86.33     1.0   
          24            0.00     88.36    -86.92       97.04     1.0   
          25            0.00    134.91    -32.25      105.16     1.0   
          26            0.00    160.31     21.17       51.26     1.0   
          27            0.00    103.29      3.51      104.69     1.0   
          28            0.00     95.56    -19.91       67.62     1.0   
          29            0.00    123.59      0.53       58.92     1.0   
          30           71.81     59.44    -70.45       95.04     1.0   
...                      ...       ...       ...         ...     ...   
     12   2             0.00    257.51     30.38       63.67     1.0   
          3             0.00    241.11      5.95      110.20     1.0   
          4             0.00    124.13    -78.47       83.76     1.0   
          5             0.00    121.06    -67.79       62.80     1.0   
          6             0.00    142.27      8.78       69.06     1.0   
          7             0.00    226.51     29.07       34.10     1.0   
          8             0.00    277.81     54.18       59.35     1.0   
          9            18.62    285.69     83.79       94.59     1.0   
          10           80.90    224.84    124.55       54.67     1.0   
          11            0.82    144.96    -31.99       84.81     1.0   
          12            0.00    120.68    -55.17       25.12     1.0   
          13            0.00    158.95    -18.99       85.84     1.0   
          14            0.00    196.51    -31.75       78.96     1.0   
          15            0.33    140.14    -50.58       41.75     1.0   
          16           20.20     86.63    -28.70       13.98     1.0   
          17            0.54     75.02    -18.20      104.48     1.0   
          18            0.00    103.88    -78.17       46.59     1.0   
          19            0.00    174.29     -0.33       55.41     1.0   
          20            0.00    162.59    -88.60      124.44     1.0   
          21            0.00    -18.39   -152.33      108.67     1.0   
          22           27.73      4.21   -162.67       69.86     1.0   
          23           13.00     38.00   -111.37       34.21     1.0   
          24            4.52     89.74    -37.01       

In [66]:
basins = get_basin_list()
forcings = {}
discharge = {}
for basin in basins:
    forcings[basin] = datautils.load_forcing(CAMELS_DIR, basin)
    discharge[basin] = datautils.load_discharge(CAMELS_DIR, basin, forcings[basin][1])

In [70]:
forcings['02046000'][0]
new=pd.merge(forcings['02046000'][0],final,on=['Year', 'Mnth', 'Day'])
new['diffmax']=new['tmax(C)']-new['wmax']
new['diffmin']=new['tmin(C)']-new['wmin']
new
#new[(new['diffmax']>0.1) | (new['diffmax']<-0.1)]
#new[(new['diffmin']>0.1) | (new['diffmin']<-0.1)]

,Year,Mnth,Day,Hr,dayl(s),prcp(mm/day),srad(W/m2),swe(mm),tmax(C),tmin(C),...,precipitation,max_temp,min_temp,wind_speed,weight,wmin,wmax,wprec,diffmax,diffmin
0,1980,1,1,12,34214.40,0.00,276.87,0.0,10.49,-4.15,...,0.00,125.25,-48.83,61.84,1.0,-4.147990,10.486332,0.000000,0.003668,-0.002010
1,1980,1,2,12,34214.40,0.00,274.74,0.0,9.70,-4.28,...,0.00,116.03,-50.63,107.10,1.0,-4.284427,9.698375,0.000000,0.001625,0.004427
2,1980,1,3,12,34214.40,0.00,285.78,0.0,8.49,-6.92,...,0.00,101.76,-82.19,44.90,1.0,-6.924640,8.486196,0.000000,0.003804,0.004640
3,1980,1,4,12,34214.40,7.84,206.80,0.0,6.06,-5.23,...,120.88,72.65,-60.15,51.61,1.0,-5.233475,6.063011,9.901695,-0.003011,0.003475
4,1980,1,5,12,34296.74,19.98,136.14,0.0,3.34,-3.45,...,261.09,28.22,-40.66,93.35,1.0,-3.448666,2.286323,22.626581,1.053677,-0.001334
5,1980,1,6,12,34560.00,5.60,240.47,0.0,1.59,-9.64,...,10.71,19.42,-114.32,90.32,1.0,-9.644204,1.590244,1.124358,-0.000244,0.004204
6,1980,1,7,12,34560.00,0.73,254.83,0.0,4.52,-9.66,...,6.82,54.34,-113.16,54.40,1.0,-9.661663,4.515592,0.625194,0.004408,0.001663
7,1980,1,8,12,34560.00,0.13,188.62,0.0,5.82,-0.24,...,0.00,71.10,-2.40,24.25,1.0,-0.236107,5.818453,0.000000,0.001547,-0.003893
8,1980,1,9,12,34560.00,0.00,177.54,0.0,6.25,0.62,...,0.00,75.10,7.85,36.23,1.0,0.623555,6.253905,0.000000,-0.003905,-0.003555
9,1980,1,10,12,34560.00,0.00,280.03,0.0,4.40,-6.36,...,0.00,53.32,-75.18,75.38,1.0,-6.355159,4.395232,0.000000,0.004768,-0.004841


In [68]:
jan=in1980[(in1980['Mnth']==12)]
jan[(jan['Day']==29)]

,Year,Mnth,Day,precipitation,max_temp,min_temp,wind_speed,weight,wmin,wmax,wprec
11685,1980,12,29,0.50,5.10,-0.40,6.07,0.303018,-0.121207,1.545390,0.151509
11685,1980,12,29,0.52,5.09,-0.27,5.84,0.000000,-0.000000,0.000000,0.000000
11685,1980,12,29,0.68,6.22,0.60,5.97,0.078875,0.047325,0.490600,0.053635
11685,1980,12,29,0.28,5.33,-0.64,6.36,0.008472,-0.005422,0.045154,0.002372
11685,1980,12,29,0.65,5.75,0.26,6.02,0.385935,0.100343,2.219127,0.250858
11685,1980,12,29,0.32,5.28,-0.57,6.33,0.074046,-0.042206,0.390963,0.023695
11685,1980,12,29,0.55,6.10,0.30,6.27,0.012134,0.003640,0.074015,0.006674
11685,1980,12,29,0.60,6.28,1.39,5.67,0.000000,0.000000,0.000000,0.000000
11685,1980,12,29,0.62,5.73,0.72,5.75,0.000000,0.000000,0.000000,0.000000
11685,1980,12,29,0.50,6.43,1.31,5.59,0.000000,0.000000,0.000000,0.000000


In [ ]:
from __future__ import print_function
import torch
x = torch.rand(5, 3)